In [1]:
import os; work_dir = "../../"; os.chdir(work_dir); work_dir = os.getcwd(); print("Current working directory:", work_dir)

Current working directory: /home/wilf/Desktop/python_workspace/PCD_SM-Fluid-Pred


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from results.data import multiple_GT_loading, make_results_dicts, load_multiple_results

loadKwargs = dict(storage_path="./storage", dataset_name="turbulent_radiative_layer_2D")

IDs = ['y2025_m04_d27_23h_30m_37s', 'y2025_m04_d02_09h_25m_13s', 'y2025_m04_d28_03h_18m_54s',
       'y2025_m04_d24_10h_54m_08s', 'y2025_m04_d24_16h_28m_53s', 'y2025_m04_d24_19h_19m_14s']


# WARNING: order of iteration must match IDs order
method_strings = ["E_loss_False__ODE_False", "E_loss_True__ODE_False"]
regularization = ["regFalse", "regTrue"]
sde_names = ["vpsde", "subvpsde", "vesde"]


test_cases = [1,2,3]

# ORDER (fields): velocity (x,y), density, pressure, ORDER (params): mach
mean = np.array([3.4847e+01, 9.4475e-01, 6.1707e-03, -2.4651e-02], dtype=np.float32)
std =  np.array([4.4284e+01, 6.0970e-02, 4.1764e-02, 4.0095e-02], dtype=np.float32)

dataKwargs = dict(test_cases=test_cases, means=mean, stds=std, dataset_name="turbulent_radiative_layer_2D")
GTs, fields_name = multiple_GT_loading(any_method_string=method_strings[0], any_ID=IDs[0], any_version=1, **dataKwargs)
GT_1, GT_2, GT_3 = GTs[1], GTs[2], GTs[3]

# Form: test_set = {"GT": {"pred": GT_1}, "regFalse": {sde: {} for sde in sde_names}, "regTrue": {sde: {} for sde in sde_names}}
empty_test_sets = make_results_dicts(GTs, regularization, sde_names, test_cases)  # test_set_1, ... = empty_test_sets[1], ...

In [3]:
import copy
print(fields_name)
fields_name[2], fields_name[3] = 'velocity ' f"$u$", 'velocity ' f"$v$"
fields_name[0], fields_name[1] = 'density ' f'$\\rho$', 'pressure ' f'$P$'
fields_name_non_latex = copy.deepcopy(fields_name)
fields_name_latex = copy.deepcopy(fields_name)
fields_name_non_latex[2], fields_name_non_latex[3] = "u", "v"
fields_name_non_latex[0], fields_name_non_latex[1] = '\\rho', 'P'
fields_name_latex[2], fields_name_latex[3] = f"$u$", f"$v$"
fields_name_latex[0], fields_name_latex[1] = f'$\\rho$', f'$P$'
print(fields_name)

['density', 'pressure', 'velocity_x', 'velocity_y']
['density $\\rho$', 'pressure $P$', 'velocity $u$', 'velocity $v$']


In [4]:
versions = [1,2,3,4,5]  # Add as many samples as you can

dataKwargs = dict(means=mean, stds=std, dataset_name="turbulent_radiative_layer_2D", mask=(mask:=GTs["mask"]))
casesKwargs = dict(empty_test_sets=empty_test_sets, versions=versions, test_cases=test_cases)
test_sets = load_multiple_results(method_strings, IDs, regularization, sde_names, **casesKwargs, **dataKwargs)

test_set_1, test_set_2, test_set_3 = test_sets[1], test_sets[2], test_sets[3]  # e.g. test_set_1["regTrue"]["vpsde"]["pred"][42]

In [5]:
from results.operations import GT_measures, pred_measures

n_bins = 200

GT_measures(test_set_1, fields_name, n_bins, dataset_name="turbulent_radiative_layer_2D")
GT_measures(test_set_2, fields_name, n_bins, dataset_name="turbulent_radiative_layer_2D")
GT_measures(test_set_3, fields_name, n_bins, dataset_name="turbulent_radiative_layer_2D")

perona=False

for version in versions:  # e.g. test_set_1["regTrue"]["vpsde"]["spatial_histogram"][42][2], the 3rd field
    pred_measures(test_set_1, version, regularization, sde_names, fields_name, n_bins, perona=perona, dataset_name="turbulent_radiative_layer_2D")
    pred_measures(test_set_2, version, regularization, sde_names, fields_name, n_bins, perona=perona, dataset_name="turbulent_radiative_layer_2D")
    pred_measures(test_set_3, version, regularization, sde_names, fields_name, n_bins, perona=perona, dataset_name="turbulent_radiative_layer_2D")

# After key measure is input, input [version] except for GT

/home/wilf/anaconda3/envs/SM-Fluid-Pred/lib/python3.10/site-packages/numpy/lib/_histograms_impl.py:901: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/home/wilf/Desktop/python_workspace/PCD_SM-Fluid-Pred/results/operations.py:364: RuntimeWarning: invalid value encountered in cast
  dict_in["GT"]["spatial_histogram"][field][t, :] = hist_counts
/home/wilf/Desktop/python_workspace/PCD_SM-Fluid-Pred/results/operations.py:627: RuntimeWarning: invalid value encountered in cast
  dict_in[regu][sde]["spatial_histogram"][version][field][t, :] = hist_counts


In [10]:
import numpy as np
import math
from itertools import product

def _flatten(d):
    """Return a 1‑D np.array no matter if *d* is list/array or dict."""
    if isinstance(d, dict):
        return np.concatenate([np.ravel(v) for v in d.values()])
    return np.ravel(d)

def _mean_var(d):
    d = _flatten(d)
    return d.mean(), d.var(ddof=1)

def _scientific_exponent(vals):
    mags = [abs(v) for v in vals if v != 0]
    if not mags:
        return 0
    exp = int(math.floor(math.log10(max(mags))))
    # keep mantissas in [0.01,100)
    if max(mags) / 10**exp < 0.1:
        exp -= 1
    return exp

def build_latex_table(
        test_cases,
        sde_names,                     # iterable
        regularization,                # iterable of True/False (or anything hashable)
        measures,                      # iterable of keys
        measure_labels,                # len == len(measures)
        versions,                      # iterable
        *,
        sde_labels=None,               # mapping or iterable aligned w/ sde_names
        regu_labels=None,              # mapping or iterable aligned w/ regularization
        highlight_lowest=None,         # list[bool] same length as measures
        num_decimals=2):

    # ------- label helpers -------
    if sde_labels is None:
        sde_labels = {k: str(k) for k in sde_names}
    elif not isinstance(sde_labels, dict):
        sde_labels = {k: lbl for k, lbl in zip(sde_names, sde_labels)}

    if regu_labels is None:
        regu_labels = {k: str(k) for k in regularization}
    elif not isinstance(regu_labels, dict):
        regu_labels = {k: lbl for k, lbl in zip(regularization, regu_labels)}

    if highlight_lowest is None:
        highlight_lowest = [True] * len(measures)

    # ---------- aggregation ----------
    result = {}
    for reg, sde in product(regularization, sde_names):
        rows = []
        for m in measures:
            cols = []
            for tc in test_cases:
                data = [_flatten(tc[reg][sde][m][v]) for v in versions]
                data = np.concatenate(data)
                cols.append(_mean_var(data))
            rows.append(cols)
        result[(reg, sde)] = rows

    # ---------- exponents per measure ----------
    exponents = []
    for mi, m in enumerate(measures):
        vals = [result[(r, s)][mi][ti][0]
                for r in regularization for s in sde_names
                for ti in range(len(test_cases))]
        exponents.append(_scientific_exponent(vals))

    # ---------- collect scaled means (for boldface) ----------
    n_cols = len(test_cases) * len(measures)
    column_values = [[] for _ in range(n_cols)]
    for reg, sde in product(regularization, sde_names):
        for mi in range(len(measures)):
            exp = exponents[mi]
            for ti in range(len(test_cases)):
                mean = result[(reg, sde)][mi][ti][0] / 10**exp
                column_values[mi*len(test_cases)+ti].append(mean)

    # choose minima or maxima per measure
    column_best = []
    for mi in range(len(measures)):
        for ti in range(len(test_cases)):
            col_idx = mi*len(test_cases)+ti
            if highlight_lowest[mi]:
                column_best.append(min(column_values[col_idx]))
            else:
                column_best.append(max(column_values[col_idx]))

    # ---------- LaTeX assembly ----------
    def fmt(mean, var, exp):
        mean_s = f"{mean/10**exp:.{num_decimals}f}"
        var_s  = f"{var/10**exp:.{num_decimals}f}"
        return f"{mean_s} $\\pm$ {var_s}"

    col_format = "|c|c|" + "|".join("c"*len(test_cases) for _ in measures) + "|"

    lines = [f"\\begin{{tabular}}{{{col_format}}}",
             "\\hline",
             "\\textbf{SDE type} & \\textbf{Regularization} & " +
             " & ".join(
                 f"\\multicolumn{{{len(test_cases)}}}{{c|}}{{{lbl} ($10^{{{exp}}}$)}}"
                 for lbl, exp in zip(measure_labels, exponents)
             ) + " \\\\"]

    # clines (first block already done)
    running = 3
    for i in range(len(measures)):
        start = running
        end   = running + len(test_cases) - 1
        lines.append(f"\\cline{{{start}-{end}}}")
        running += len(test_cases)

    # table body
    for reg in regularization:
        for sde in sde_names:
            row_cells = []
            for mi in range(len(measures)):
                exp = exponents[mi]
                for ti in range(len(test_cases)):
                    mean, var = result[(reg, sde)][mi][ti]
                    scaled_mean = mean / 10**exp
                    col_idx = mi*len(test_cases)+ti
                    cell = fmt(mean, var, exp)
                    if math.isclose(scaled_mean, column_best[col_idx]):
                        cell = f"\\textbf{{{cell}}}"
                    row_cells.append(cell)

            lines.append(f"{sde_labels[sde]} & {regu_labels[reg]} & "
                         + " & ".join(row_cells) + " \\\\")
        lines.append("\\hline")

    lines.append("\\end{tabular}")
    return "\n".join(lines)

In [9]:
test_cases = [test_set_1, test_set_2, test_set_3]


# latex = build_latex_table(
#     test_cases       = test_cases,
#     sde_names        = ["vpsde", "subvpsde", "vesde"],
#     regularization   = regularization,
#     measures         = [r"MSE $\downarrow$", r"pearson $\uparrow$", "wasserstein_distance", "KL_divergence"],
#     measure_labels   = ["MSE", "pearson", "W1", "KL"],
#     versions         = versions
# )


latex = build_latex_table(
    test_cases       = test_cases,
    sde_names        = ["vpsde", "subvpsde", "vesde"],
    regularization   = regularization,
    measures         = ["MSE",
                        "RE",
                        "pearson",
                        "KL_divergence"],
    measure_labels   = ["$MSE \downarrow$",
                        "$Re$",
                        "pearson $\\uparrow$",
                        "$\\bar{\mathbb{D}}_{KL}$"],
    versions         = versions,
    sde_labels       = ["VP","sub‑VP","VE"],
    regu_labels      = {regularization[0]:"False", regularization[1]:"True"},
    highlight_lowest = [True, True, False, True]   # MSE ↓, FID ↑, PSNR ↓
)

print(latex)

\begin{tabular}{|c|c|ccc|ccc|ccc|ccc|}
\hline
\textbf{SDE type} & \textbf{Regularization} & \multicolumn{3}{c|}{$MSE \downarrow$ ($10^{2}$)} & \multicolumn{3}{c|}{$Re$ ($10^{-1}$)} & \multicolumn{3}{c|}{pearson $\uparrow$ ($10^{-1}$)} & \multicolumn{3}{c|}{$\bar{\mathbb{D}}_{KL}$ ($10^{1}$)} \\
\cline{3-5}
\cline{6-8}
\cline{9-11}
\cline{12-14}
VP & False & \textbf{0.66 $\pm$ 212.00} & 0.77 $\pm$ 275.67 & 0.69 $\pm$ 231.45 & \textbf{3.27 $\pm$ 0.56} & \textbf{3.48 $\pm$ 0.56} & 3.51 $\pm$ 0.81 & \textbf{9.52 $\pm$ 0.01} & 9.45 $\pm$ 0.01 & 9.50 $\pm$ 0.01 & 1.05 $\pm$ 11.18 & 1.11 $\pm$ 10.85 & 0.90 $\pm$ 9.67 \\
sub‑VP & False & 0.74 $\pm$ 271.05 & 1.00 $\pm$ 457.83 & \textbf{0.58 $\pm$ 158.65} & 3.69 $\pm$ 0.72 & 4.06 $\pm$ 0.80 & \textbf{3.23 $\pm$ 0.71} & 9.48 $\pm$ 0.01 & 9.30 $\pm$ 0.02 & \textbf{9.59 $\pm$ 0.01} & 1.24 $\pm$ 11.79 & 1.32 $\pm$ 11.45 & 0.93 $\pm$ 10.58 \\
VE & False & 0.96 $\pm$ 408.64 & 1.02 $\pm$ 456.74 & 0.97 $\pm$ 413.27 & 4.05 $\pm$ 0.75 & 4.16 $\pm$ 0.77 & 